# Blind Source Separation of EEG Signals through PMF


In [ ]:
import numpy as np
import mne
mne.set_config('MNE_BROWSER_BACKEND', 'matplotlib')
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from pmf import *
from utils import * 

# EEG Import

In [ ]:
raw = mne.io.read_raw_eeglab(input_fname = "data.set")
raw.info

In [ ]:
raw.plot(n_channels=len(raw));

# X = WZ

X is regarded as mixtures, which is an M by N matrix.

W represents a linear mapping, which is full column rank M by r matrix. 

Z represents source signals. It is r by N matrix.

In [ ]:
X =raw[:][0]
r = 32 # Number of sources
M,N = X.shape

In [ ]:
events, event_id = mne.events_from_annotations(raw)
tmin = -0.01  # Start time before event
tmax = 2.5 # End time after event
epochs = mne.Epochs(raw, events, event_id=2, tmin=tmin, tmax=tmax, preload=True)
avg_data = epochs.average().data;

In [ ]:
#PMF EXECUTION FOR ALL DATA

W, Z = PMF(raw, avg_data, r = r, NumberofIterations = 3000, muv=10**(9.), epsv=1e-9, tuning=False,tmin = tmin, tmax = tmax)

In [ ]:
#Calculate the RVs

dip_pmf_rvs, dip_pmf_gofs = calculate_rvs(raw,W,M,N,r,tmin = tmin, tmax = tmax)


In [ ]:
#Threshold for the RVs to the purpose of plotting 
threshold = 0.15

In [ ]:
events, event_id = mne.events_from_annotations(raw)

ch_names = []
for i in range(r):
    ch_names.append(f"PMF {i}")

ch_types = ["eeg"] * r
Z_info = mne.create_info(ch_names, ch_types=ch_types, sfreq=1024)
Z_mne = mne.io.RawArray(Z, Z_info)

# Square epochs
square_epochs = mne.Epochs(Z_mne, events, event_id=2, tmin=tmin, tmax=tmax, preload=True) 

npArray_dip_pmf_rvs = np.array(dip_pmf_rvs)

cmap = plt.get_cmap('tab20') 


indices_of_thresholded_sources = [i for i, x in enumerate(npArray_dip_pmf_rvs) if x < threshold]

# For plotting purposes

for i in indices_of_thresholded_sources:
    plt.plot(Z[i], label=f'Source {i}')


plt.legend()

plt.show()

# Thresholded Topomaps


In [ ]:
#Plot
pmf_plot(r, W, raw, dip_pmf_rvs,threshold)